In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AI_Lec_run

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install docx
!pip install exceptions
!pip install python-docx

In [ ]:
"""
이 코드는 텍스트 파일을 처리하기 위한 파이썬 스크립트입니다.

사용법:
1. 환경 변수 설정: .env 파일에 OpenAI API 키를 설정하세요. "OPEN_API_KEY=your-api-key" 형식을 따릅니다.
2. raw_root_dir 설정: 'raw_root_dir' 변수에 원본 텍스트 파일이 위치한 루트 디렉토리의 경로를 문자열로 설정합니다.
   예) raw_root_dir = 'C:\\Users\\username\\Documents\\raw_texts'

요구되는 파일 포맷:
- 파일은 .txt 확장자를 가져야 합니다.
- 파일명은 'raw_subject_week_day_numb.txt' 형태를 따라야 합니다. 각 항목은 다음과 같습니다.
  - raw: 파일이 원시 텍스트임을 나타냅니다. 이 접두사가 없는 파일은 무시됩니다.
  - subject: 과목명입니다. 예) "math", "physics", "history"
  - week: 해당 주차를 나타냅니다. 예) "week1", "week2"
  - day: 해당 일자를 나타냅니다. 예) "day1", "day2"
  - numb: 번호를 나타냅니다. 예) "1", "2"

코드 작동 방식:
1. get_txt_files 함수는 raw_root_dir에서 모든 .txt 파일의 전체 경로를 찾습니다.
2. raw_text_to_product 함수는 각 텍스트 파일을 처리하고, 이를 여러 형태로 변환합니다.

파일 저장 위치:
- 중간 결과물은 'intermediate' 디렉토리에 'subject_week_day_numb_paragraphed.txt' 형태로 저장됩니다.
- 교정된 텍스트는 'intermediate' 디렉토리에 'subject_week_day_numb_corrected.txt' 형태로 저장됩니다.
- 영어로 번역된 텍스트는 'englished' 디렉토리에 'subject_week_day_numb_englished.txt' 형태로 저장됩니다.
- 요약된 텍스트는 'summary' 디렉토리에 'subject_week_day_numb_summarised.txt' 형태로 저장됩니다.

디렉토리가 없을 경우 자동으로 생성됩니다.
"""
# 코드 시작

from correct_gpt_v3 import *
from dotenv import load_dotenv
import os
import shutil

# Initialize environment variables
load_dotenv()
openai.api_key = os.getenv("OPEN_API_KEY")

raw_root_dir = '/content/drive/MyDrive/raw' # Update this with your actual root directory path
used_raw_root_dir = '/content/drive/MyDrive/used_raw'

def get_txt_files(root_dir):
    print("Scanning for TXT files...")
    txt_files = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.txt'):
                full_path = os.path.join(root, file)
                txt_files.append(full_path)
    print(f"Found {len(txt_files)} TXT files.")
    return txt_files


def raw_text_to_product(raw_text_path, subject, week, day, numb):
    print(f"Processing: {raw_text_path}")
    base_name = f'{subject}_{week}_{day}_{numb}'

    for dir_name in ['intermediate', 'englished', 'summary']:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

    print("Converting to paragraphed text...")
    to_paragraphed(
        original_file_path=raw_text_path,
        output_file_path=os.path.join('intermediate', f'{base_name}_paragraphed.txt')
    )

    print("Correcting the text...")
    to_corrected(
        original_file_path=os.path.join('intermediate', f'{base_name}_paragraphed.txt'),
        output_file_path=os.path.join('intermediate', f'{base_name}_corrected.txt')
    )

    print("Translating to English...")
    to_englished(
        original_file_path=os.path.join('intermediate', f'{base_name}_corrected.txt'),
        output_file_path=os.path.join('englished', f'{base_name}_englished.txt')
    )

    print("Creating summary...")
    to_summarised(
        original_file_path=os.path.join('englished', f'{base_name}_englished.txt'),
        output_file_path=os.path.join('summary', f'{base_name}_summarised.txt')
    )
    print(f"Processing for {raw_text_path} to word file")
    convert_markdown_to_word(os.path.join('summary', f'{base_name}_summarised.txt'), os.path.join('summary', f'{base_name}_word'))

    print(f"Processing for {raw_text_path} completed.")
    shutil.move(raw_text_path, os.path.join(used_raw_root_dir, f'raw_{base_name}.txt'))
    print(f"{raw_text_path} moved to {os.path.join(used_raw_root_dir, f'raw_{base_name}.txt')}")


def extract_info_from_file_name(file_name):
    file_name = file_name[:-4]
    components = file_name.split('_')
    if components[0] == 'raw':
        _, subject, week, day, numb = components
        return subject, week, day, numb
    else:
        return None, None, None, None


def full_function():
    raw_text_file_list = get_txt_files(raw_root_dir)
    for file_path in raw_text_file_list:
        file_name = os.path.basename(file_path)
        print(file_name)
        subject, week, day, numb = extract_info_from_file_name(file_name)
        print(subject, week, day, numb)
        if subject is not None:
            raw_text_to_product(raw_text_path=file_path, subject=subject, week=week, day=day, numb=numb)

error_count = 0

while True:
    try:
        print('starting full function')
        full_function()
        break
    except Exception as e:
        print(f"Error: {e}")
        error_count += 1
        if error_count >= 5:
            print("Too many errors. Exiting.")
            break
        print('retrying')

